In [4]:
import numpy as np
import pandas as pd
from data_process import data_process
from fit_logistic import fit_logistic,accuracy,loss,forward
df=pd.read_csv('train.csv')

# Create 'total_call' feature
df['total_call'] = df['total_day_calls'] + df['total_eve_calls'] + df['total_night_calls']

# Create 'total_charges' feature
df['total_charges'] = df['total_day_charge'] + df['total_eve_charge'] + df['total_night_charge']

# Create 'total_minutes' feature
df['total_minutes'] = df['total_day_minutes'] + df['total_eve_minutes'] + df['total_night_minutes']
df = df.drop(['total_day_calls', 'total_eve_calls', 'total_night_calls'], axis=1)

# Delete contributing features for 'total_charges'
df = df.drop(['total_day_charge', 'total_eve_charge', 'total_night_charge'], axis=1)

# Delete contributing features for 'total_minutes'
df = df.drop(['total_day_minutes', 'total_eve_minutes', 'total_night_minutes'], axis=1)
# Convert DataFrame to numpy array
data = np.array(df)

# Update feature columns and list
features_columns = df.columns[:-1]
features_list = [col for col in features_columns if col not in ['state', 'area_code', 'account_length']]

X_train, y_train, X_valid, y_valid, X_test, y_test = data_process(data, 0.7, 0.15, 0.15, features_columns, features_list)

X_train.shape

(2975, 10)

In [128]:
#driver code for the project
import numpy as np
import pandas as pd

df=pd.read_csv('train.csv')

data=np.array(df)
np.random.shuffle(data)
def one_hot(X,column_names):
    features_dict = {}
    m,n=X.shape
    X_new = np.empty((m,0))
    index=0
    for i in X[0,:]:
        if isinstance(i,str):
            uniques, ids= np.unique(X[:,index], return_inverse=True)
            one_hot_temp = np.zeros((m, len(uniques)))
            one_hot_temp[np.arange(m), ids] = 1
            features_dict[column_names[index]] = one_hot_temp
            X_new=np.concatenate((X_new,one_hot_temp),axis=1)
        else:
            features_dict[column_names[index]] = X[:, index].reshape(-1, 1)
            X_new= np.concatenate((X_new,X[:,index].reshape(-1,1)),axis=1)
        index+=1
    return X_new,features_dict
data[data=='no']=0.
data[data=='yes']=1.
X = data[:, :-1]
y = data[:, -1]
features_columns = df.columns[:-1]
X, features_dict = one_hot(X, features_columns)
y=y.astype(float)
X=X.astype(float)

features_dict['area_code'].shape

def feature_selection(features_dict, feature_names):
    X_features=np.zeros((features_dict[feature_names[0]].shape[0],0))
    for feature in feature_names:
        X_features=np.concatenate((X_features,features_dict[feature]),axis=1)
           
    return X_features
features_columns_excluding_state_and_area_code = [col for col in features_columns if col not in ['state', 'area_code', 'account_length']]
X = feature_selection(features_dict, features_columns_excluding_state_and_area_code)

def normalize(data,features):
    for i in features:
        data[:,i] = (data[:,i] - data[:,i].mean()) / data[:,i].std()
    
    return data

X = normalize(X, np.arange(X.shape[1]))

split_train = int(0.6 * X.shape[0])
split_valid= int(0.2 * X.shape[0])
split_test = int(0.2 * X.shape[0])

X_train = X[:split_train]
y_train = y[:split_train]
X_valid = X[split_train:split_train+split_valid]
y_valid = y[split_train:split_train+split_valid]
X_test = X[split_train+split_valid:]
y_test = y[split_train+split_valid:]

def init_weight_and_bias(n):
    return np.random.randn(n), np.random.randn(1)



def sigmoid(x):
    return 1/(1+np.exp(-x))

def forward(X,W,b):
   
    A=np.dot(X,W)+b
    A=A.astype(float)
    return sigmoid(A)
def loss(y,y_hat):
    return -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))

def train(X, y, W, b, lr=0.01, epochs=1000, reg=0.01):
    costs=[]
    for epoch in range(epochs):
        
            # Forward pass
        
            y_hat = forward(X, W, b)
            
            # Loss calculation
            l = loss(y, y_hat)
            costs.append(l)
            print(f"Epoch {epoch+1}, Loss: {l}")

            # Gradient calculation
            print(X.T.shape)
            dW = np.dot(X.T, (y_hat - y)) / X.shape[0] + reg * W
          
            db = np.mean((y_hat - y))

            # Parameter updates
            dW=dW.astype(float)
            W -= lr * dW
            b -= lr * db

    return W, b,costs

def accuracy(W,b,X,y):
    y_hat = forward(X, W, b)
    y_hat = np.round(y_hat)
    return np.mean(y_hat == y)

def fit_logistic(X, y, lr=0.01, epochs=1000, reg=0.01):
    W, b = init_weight_and_bias(X.shape[1])
    
    W, b, costs = train(X, y, W, b, lr, epochs, reg)
    return W, b, costs


lr_value=[0.01,0.03,0.06,0.1,0,2,0.5,1]
reg_value=[0.01,0.03,0.06,0.1,0,2,0.5,1]
max_accuracy=0
with open('output.txt', 'w') as f:
    pass
for lr in lr_value:
    
   
   with open('output.txt', 'a') as f:
        f.write(f"Training with lr={lr}\n")
      
        f.write("-------------------------------------------------\n")
        W, b, costs = fit_logistic(X_train, y_train, lr, 10000, 0.01)
        y_hat = forward(X_valid, W, b)

        f.write('\n')
        f.write(f"Validation Loss: {loss(y_valid, y_hat)}\n")
        f.write(f"Validation Accuracy: {accuracy(W, b, X_valid, y_valid)}\n")
        if accuracy(W, b, X_valid, y_valid)>max_accuracy:
            W_best_fit=W
            b_best_fit=b
            costs_best_fit=costs
            
        f.write("Test loss:{}\n".format(loss(y_test, forward(X_test, W, b))))
        f.write("Test accuracy:{}\n".format(accuracy(W, b, X_test, y_test)))
      
        f.write("-------------------------------------------------\n")
        f.write("\n")
    
f.close()

for reg in reg_value:
    
   
   with open('output.txt', 'a') as f:
        f.write(f"Training with reg={reg}\n")
      
        f.write("-------------------------------------------------\n")
        W, b, costs = fit_logistic(X_train, y_train, 0.01, 10000, reg)
        y_hat = forward(X_valid, W, b)

        f.write('\n')
        f.write(f"Validation Loss: {loss(y_valid, y_hat)}\n")
        f.write(f"Validation Accuracy: {accuracy(W, b, X_valid, y_valid)}\n")
        if accuracy(W, b, X_valid, y_valid)>max_accuracy:
            W_best_fit=W
            b_best_fit=b
            costs_best_fit=costs
            
        f.write("Test loss:{}\n".format(loss(y_test, forward(X_test, W, b))))
        f.write("Test accuracy:{}\n".format(accuracy(W, b, X_test, y_test)))
      
        f.write("-------------------------------------------------\n")
        f.write("\n")
    
f.close()

np.savez('weights.npz', W=W_best_fit, b=b_best_fit,costs=costs_best_fit)




    
    

Epoch 1, Loss: 1.3957484824134703
(16, 2550)
Epoch 2, Loss: 1.3917208057481374
(16, 2550)
Epoch 3, Loss: 1.3877010090476296
(16, 2550)
Epoch 4, Loss: 1.3836891156929265
(16, 2550)
Epoch 5, Loss: 1.3796851490828552
(16, 2550)
Epoch 6, Loss: 1.375689132632306
(16, 2550)
Epoch 7, Loss: 1.371701089770435
(16, 2550)
Epoch 8, Loss: 1.3677210439388392
(16, 2550)
Epoch 9, Loss: 1.3637490185897103
(16, 2550)
Epoch 10, Loss: 1.3597850371839613
(16, 2550)
Epoch 11, Loss: 1.3558291231893407
(16, 2550)
Epoch 12, Loss: 1.351881300078516
(16, 2550)
Epoch 13, Loss: 1.347941591327138
(16, 2550)
Epoch 14, Loss: 1.344010020411888
(16, 2550)
Epoch 15, Loss: 1.3400866108084961
(16, 2550)
Epoch 16, Loss: 1.33617138598974
(16, 2550)
Epoch 17, Loss: 1.3322643694234286
(16, 2550)
Epoch 18, Loss: 1.3283655845703601
(16, 2550)
Epoch 19, Loss: 1.324475054882256
(16, 2550)
Epoch 20, Loss: 1.320592803799682
(16, 2550)
Epoch 21, Loss: 1.3167188547499453
(16, 2550)
Epoch 22, Loss: 1.312853231144967
(16, 2550)
Epoch 2

KeyboardInterrupt: 

In [124]:
X_test=np.ones((71,2000))
y_test=np.ones((2000,1))
dW = np.dot(X_test,y_test) 
          
print(dW.shape)

(71, 1)
